<a href="https://colab.research.google.com/github/pycriador/PyCSVtoXLSX/blob/main/%5BUpload%5D_PyJoinCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Projeto de Março

Um programa para unificar vários arquivos CSV em um único arquivo Excel (XLSX), utilizando o modo tabela do próprio Excel

Primeiro vamos instalar a biblioteca responsável por exportar em formato de tabela

In [ ]:
!pip install xlsxwriter

Essa é a função principal, você precisa só executar por enquanto. Se quiser modificar a codificação do seu arquivo CSV ou o separador, será necessário executar novamente essa parte.

In [ ]:
import pandas as pd
import numpy as np
import time
import os, re
import xlsxwriter

class appCSV():

    def __init__(self):
        self.df_base = []

    def files(self, path, extension):
        for file in os.listdir(path):
            if os.path.isfile(os.path.join(path, file)):
                if re.search(extension, file):
                    yield file

    def read(self, input=None, clean_cache=None):
        if clean_cache:
            self.df_base = []

        df = pd.read_csv(input, sep=";", encoding='iso-8859-1', low_memory=False)

        base_cols = df.columns.tolist()
        for col in base_cols:
            try:
                df[col] = df[col].str.strip()
            except Exception as e:
                pass
        self.df_base.append(df)

    def join(self):
        self.result = pd.concat(self.df_base)
        self.result = self.result.replace(np.nan, '', regex=True)


    def export(self, output='OutputFile', timestamp=True, sheet='Base'):
        def colToExcel(col):  # col is 1 based
            excelCol = str()
            div = col
            while div:
                (div, mod) = divmod(div - 1, 26)  # will return (x, 0 .. 25)
                excelCol = chr(mod + 65) + excelCol

            return excelCol

        if timestamp:
            timestamp = int(time.time())
            timestamp = f' - {timestamp}'
        else:
            timestamp = ''

        #Output sem formatação
        #with pd.ExcelWriter(f'PANDAS_{output}{timestamp}.xlsx') as writer:
        #    self.result.to_excel(writer, sheet_name=sheet, index=False)

        #Output com formatação e tabela do Excel
        workbook = xlsxwriter.Workbook(f'{output}{timestamp}.xlsx', options={'nan_inf_to_errors': True})
        # Criar aba
        worksheet = workbook.add_worksheet(sheet)
        # Set the columns widths
        worksheet.set_column('A:Z', 15)
        workbook.formats[0].set_font_size(8)
        colunas = self.result.columns.tolist()
        n_l = ['header' for x in colunas]
        lines = len(self.result.index) + 1

        columns = []
        for x in colunas:
            for y in n_l:
                columns.append({y: x})
                break

        col = colToExcel(len(colunas))
        worksheet.add_table('A1:%s%s' % (col, lines), {'data': self.result.values.tolist(),
                                                       'columns': columns,
                                                       'style': 'Table Style Light 11',
                                                       'name': sheet})
        workbook.close()

Agora vamos realizar o upload dos arquivos e limpar os arquivos antigos

In [ ]:
from google.colab import files
import shutil

print('Escolha o arquivo CSV')
ap_up = files.upload()

print('Limpando arquivos antigos')
try:
  shutil.rmtree('arquivos')
except Exception as e:
  print(f'Erro: {e}')

!mkdir arquivos
!mv *.csv arquivos

Agora é só executar esse ultimo passo para exportar o arquivo em XLSX

In [ ]:
app = appCSV()

dir_list = []

path = ['arquivos']
extension = '.csv$'

for path_info in path:
    dirs = app.files(path_info, extension)
    for dir_info in dirs:
        dir_list.append("%s/%s" % (path_info, dir_info))

if len(dir_list) > 0:
    for csvfile in dir_list:
        print(f'Trabalhando no arquivo {csvfile}')
        app.read(input=csvfile, clean_cache=False)

    #Exportar em um arquivo tudo
    app.join()
    app.export(output='Arquivo CSV unificado', timestamp=True, sheet='Base')